In [1]:
import pandas as pd
import numpy as np 
import datetime
inventory = pd.read_csv("data/inventory.csv")
products = pd.read_csv("data/products.csv")
promotions = pd.read_csv("data/promotions.csv")
transactions = pd.read_csv("data/transactions.csv")

#create a reference dataframe
dayofyear = list(range(365))
date = [datetime.datetime(2018, 1, 1) + datetime.timedelta(day) for day in dayofyear]
df_date = pd.DataFrame({"day":dayofyear, "date":date})

import ast
for colname in inventory.columns[2:]:
    inventory[colname] = [ast.literal_eval(i) for i in inventory[colname]]
    #converting inventory rows to list of tuples from string
product = "Biologische prei"
filtered_transactions = transactions[transactions["description"] == product]
filtered_inventory = inventory[["day", "before or after delivery", product]]

In [2]:
from waste_functions import create_bb_data, add_purchases

waste_df = create_bb_data(filtered_inventory, product)
waste_df_extended = add_purchases(waste_df, transactions, product)
waste_df_extended

(101, 5)
c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src\waste_functions.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_transactions["DOY"] = filtered_transactions.apply(input_DOY, axis=1)
C:\Users\zolyo\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,amount,DOY,cumulative purchases,purchases,remaining,waste,week,waste nn
best before day,,,,,,,,
14,26,14.0,36.0,NaN,-10.0,-10.0,3,0.0
17,26,17.0,41.0,5.0,21.0,11.0,3,11.0
21,26,21.0,60.0,19.0,7.0,7.0,4,7.0
24,26,24.0,68.0,8.0,18.0,18.0,4,18.0
28,26,28.0,88.0,20.0,6.0,6.0,5,6.0
...,...,...,...,...,...,...,...,...
350,26,350.0,1229.0,11.0,15.0,15.0,51,15.0
353,26,353.0,1237.0,8.0,18.0,18.0,51,18.0
357,26,357.0,1251.0,14.0,12.0,12.0,52,12.0


## Creating range intervals for mid-delivery sales calclation

In [3]:
best_before = []
for element_list in inventory[product]:
    for batch in element_list:
        best_before.append(batch[0])
bb_dates = list(np.unique(best_before))
#list of each unique best before dates
bb_dates.insert(0, 0)
ranges = []
i = 0
for item in bb_dates:
    if i < len(bb_dates)-1:
        i += 1
        ret_tupl = (item, bb_dates[i])
        ranges.append(ret_tupl)



In [4]:
from demand import *
compl_prod="Basis anannasstukjes"
discount_per_day = pd.read_csv("nr_discounts_per_day.csv")
prepared_transactions = prepare_data(filtered_transactions, discounts_per_day=discount_per_day)
output_dct = prepare_demand_function(prepared_transactions)

c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src\demand.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_transactions["day"] = pd.to_datetime(filtered_transactions["day"], dayfirst=True)
c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src\demand.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_transactions["discount"] = 100-(
c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src\demand.py:29: SettingWith

In [5]:
model = fit_demand_function(output_dct)

Model fitted; test set metrics: MAE: 1.4358902314077937 MSE: 3.326587120366766, R^2 score: -0.11815451624990914


In [6]:
from waste_functions import predicted_demand

test = predicted_demand(waste_df_extended, ranges, model, output_dct, prepared_transactions)

as.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prep.dropna(inplace=True)
c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src\demand.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prep.dropna(inplace=True)
c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src\demand.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prep.dropna(inplace=True)
c:\Users\zolyo\OneDrive\Documents\Quartile 3\JBG050 Data Challenge 2\Code\src\demand.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [7]:
test

,amount,DOY,cumulative purchases,purchases,remaining,waste,week,waste nn,predicted demand,demand_discounts
best before day,,,,,,,,,,
14,26,14.0,36.0,NaN,-10.0,-10.0,3,0.0,38.803361,40.901873
17,26,17.0,41.0,5.0,21.0,11.0,3,11.0,7.904251,10.002763
21,26,21.0,60.0,19.0,7.0,7.0,4,7.0,12.064346,14.162858
24,26,24.0,68.0,8.0,18.0,18.0,4,18.0,12.743521,14.842033
28,26,28.0,88.0,20.0,6.0,6.0,5,6.0,22.054773,24.153285
...,...,...,...,...,...,...,...,...,...,...
350,26,350.0,1229.0,11.0,15.0,15.0,51,15.0,21.842744,23.941256
353,26,353.0,1237.0,8.0,18.0,18.0,51,18.0,12.500279,14.598791
357,26,357.0,1251.0,14.0,12.0,12.0,52,12.0,21.619144,23.717656
